# Setting Up

## Install Dependencies

In [ ]:
!pip install -U tensorflow
!pip install -U numpy
!pip install -U Pillow
!pil install -U opencv-python

## Imports

In [ ]:
import os
import time
import glob
import argparse
import numpy as np
import tensorflow as tf
import cv2
from dataturehub.hub import HubModel
from dataturehub.visualize import Visualize
from PIL import Image

## Comment out next line to use GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
## Comment out to set verbose to true
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Prediction Preparation

## Set all necessary variables

In [ ]:
input_folder = "./input/" # Path to folder that contains input images
output_folder = "./output/" # Path to folder to store predicted images
threshold = 0.7 # Prediction confidence threshold

project_secret = "<YOUR_PROJECT_SECRET_KEY>"
model_key = "<YOUR_MODEL_KEY>"

## Initialize Hub

In [ ]:
hub_model = HubModel(
    project_secret=project_secret,
    model_key=model_key,
  )

## Load the model and label map

In [ ]:
trained_model = hub_model.load_tf_model()
category_index = hub_model.load_label_map()

## Run Prediction on Each Image

In [ ]:
all_detections = []
all_images = glob.glob(os.path.join(input_folder, "*"))
for each_image in all_images:
  print("Predicting for {}...".format(each_image))
  input_tensor = hub_model.load_image(each_image)
  detections_output = trained_model(input_tensor)
  all_detections.append(detections_output)

## Save Image Visualizations

In [ ]:
for each_image, each_detection in zip(all_images, all_detections):
  visualized_image = Visualize(each_image, each_detection, category_index, threshold)
  filename = os.path.basename(each_image)
  image_predict = Image.fromarray(visualized_image)
  image_predict.save(os.path.join(output_folder, filename))

  print(
    "Saving predicted images to {}...".format(
      os.path.join(args.output, filename)
    )
  )